In [ ]:
import pandas as pd
import numpy as np

import re

In [ ]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50

# Objective
we clean the accepted loans of Lending Club https://www.kaggle.com/wordsforthewise/lending-club to prepare for EDA and machine lerning.

# Parsing the dates

In [ ]:
# dtypes = {'issue_d': 'str', 'sec_app_earliest_cr_line':'str', 
#           'earliest_cr_line':'str', 'last_pymnt_d':'str', 
#           'last_credit_pull_d':'str'}

# accepted = pd.read_csv('/Users/ivanpassoni/Downloads/lending-club/accepted_2007_to_2018Q4.csv',
#                        low_memory = False, dtype = dtypes)

### Dropping bad Id's

In [ ]:
# remove bad id's
# accepted.drop(index= accepted.loc[accepted['id'].apply(lambda s: len(str(s))) > 10, :].index, inplace = True)

### Parsing Datas

In [ ]:
# # Parsing issue_date, last_pymnt_d, earliest_cr_line, last_credit_pull_d datetime
# def parse_date(d):
#     return pd.to_datetime(d, format='%b-%Y')

# accepted['issue_d']            = accepted['issue_d'].apply(parse_date)
# accepted['last_pymnt_d']       = accepted['last_pymnt_d'].apply(parse_date)
# accepted['earliest_cr_line']   = accepted['earliest_cr_line'].apply(parse_date)
# accepted['last_credit_pull_d'] = accepted['last_credit_pull_d'].apply(parse_date)
# next_pymnt_d will be droped later

### Saved and reading the data parsed

In [ ]:
# accepted.to_csv('/Users/ivanpassoni/Downloads/lending-club/accepted_2007_to_2018Q4_parsed_data.csv', index = False)

# General data cleaning

In [ ]:
parse_dates = ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d']

accepted = pd.read_csv('/Users/ivanpassoni/Downloads/lending-club/accepted_2007_to_2018Q4_parsed_data.csv',
                       low_memory = False, parse_dates = parse_dates)

### Merging joint and individual loan applications

We consider a joint application as an individual application with the annual_inc, dti and revol_bal given by their joint values.

In [ ]:
# Considering annual_inc == annual_inc_joint, dti == dti_joint
accepted['annual_inc'] = accepted[['application_type', 'annual_inc', 'annual_inc_joint']]\
.apply(lambda row: row['annual_inc'] if row['application_type'] == 'Individual' else row['annual_inc_joint'], axis = 1)

accepted['dti'] = accepted[['application_type', 'dti', 'dti_joint']]\
.apply(lambda row: row['dti'] if row['application_type'] == 'Individual' else row['dti_joint'], axis = 1)

accepted['revol_bal'] = accepted[['application_type', 'revol_bal', 'revol_bal_joint']]\
.apply(lambda row: row['revol_bal'] if row['application_type'] == 'Individual' else row['revol_bal_joint'], axis = 1)

### Imputing Last Payment Date

We impute the last_pymnt_d by adding to issue_d the number of months given by total_pymnt/installment

In [ ]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta

def increment_months(s, length):
    while length > 0:
        if s.month == 12:
            s = s + timedelta(days = 31)
        else:
            s = s + relativedelta(months = 1)
        length -= 1
    return s

accepted['last_pymnt_d'].fillna(accepted.loc[accepted['last_pymnt_d'].isnull(), 
                                             ['issue_d', 'total_pymnt', 'installment']]\
                        .apply(lambda s: increment_months(s[0],np.floor(s[1]/s[2])), axis = 1),
                                inplace = True)

### Defining the Apriori columns and droping irrelevant columns

In [ ]:
apriori_columns = ['id', 'member_id', 'loan_amnt', 'funded_amnt', 
                   'funded_amnt_inv','term','int_rate',
                   'installment','grade','sub_grade','emp_title',
                   'emp_length','home_ownership', 'annual_inc',
                   'issue_d','url','desc','purpose','title',
                   'zip_code','addr_state','dti','delinq_2yrs',
                   'earliest_cr_line','fico_range_low',
                   'fico_range_high','inq_last_6mths',
                   'mths_since_last_delinq','mths_since_last_record',
                   'open_acc','pub_rec','revol_bal','revol_util',
                   'total_acc','initial_list_status',
                   'collections_12_mths_ex_med',
                   'mths_since_last_major_derog','application_type',
                   'annual_inc_joint','dti_joint','acc_now_delinq',
                   'tot_coll_amt','tot_cur_bal','open_acc_6m',
                   'open_act_il','open_il_12m','open_il_24m',
                   'mths_since_rcnt_il','total_bal_il','il_util',
                   'open_rv_12m','open_rv_24m','max_bal_bc','all_util',
                   'total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m',
                   'acc_open_past_24mths','avg_cur_bal','bc_open_to_buy',
                   'bc_util','chargeoff_within_12_mths','delinq_amnt',
                   'mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op',
                   'mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc',
                   'mths_since_recent_revol_delinq','num_accts_ever_120_pd',
                   'num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl',
                   'num_il_tl','num_op_rev_tl','num_rev_accts',
                   'num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m',
                   'num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m',
                   'pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies',
                   'tax_liens','tot_hi_cred_lim','total_bal_ex_mort',
                   'total_bc_limit','total_il_high_credit_limit',
                   'revol_bal_joint','sec_app_fico_range_low',
                   'sec_app_fico_range_high','sec_app_earliest_cr_line',
                   'sec_app_inq_last_6mths','sec_app_mort_acc',
                   'sec_app_open_acc','sec_app_revol_util',
                   'sec_app_open_act_il','sec_app_num_rev_accts',
                   'sec_app_chargeoff_within_12_mths',
                   'sec_app_collections_12_mths_ex_med',
                   'sec_app_mths_since_last_major_derog', 
                   'disbursement_method']

### Droping some columns

In [ ]:
# Drop irrelevant columns
hardships = [col for col in accepted.columns if 'hard' in col]
hardships.extend(['payment_plan_start_date', 'orig_projected_additional_accrued_interest'])

investors = [col for col in accepted.columns if 'inv' in col]

secundary_applicants = [col for col in accepted.columns if 'sec' in col]

settlements = [col for col in accepted.columns if 'settlement' in col]

joint = [col for col in accepted.columns if 'joint' in col]

drop_columns = ['member_id', 'url', 'desc', 
                'pymnt_plan', 'title', 
                'zip_code', 'disbursement_method',
                'next_pymnt_d', 'policy_code',
                'deferral_term', 'emp_title',
                'application_type']               +\
                hardships + settlements           +\
                secundary_applicants + investors  +\
                joint

accepted.drop(columns = drop_columns, inplace = True)

# updating the apriori columns list

apriori_columns = [col for col in apriori_columns if col not in drop_columns]

### Dropping some rows

In [ ]:
accepted.drop(index=accepted.loc[((accepted['home_ownership'] == 'ANY')|
                                 (accepted['home_ownership'] == 'OTHER')|
                                  (accepted['home_ownership'] == 'NONE')), :].index,
              inplace = True)

### Subdividing the data by year

In [ ]:
accepted_b_2015    = accepted.loc[accepted['issue_d'] < '2015-01-1', :]
accepted_2015      = accepted.loc[(accepted['issue_d'] >= '2015-01-1') & (accepted['issue_d'] < '2016-01-1'), :]
accepted_2016      = accepted.loc[(accepted['issue_d'] >= '2016-01-1') & (accepted['issue_d'] < '2017-01-1'), :]
accepted_2017      = accepted.loc[(accepted['issue_d'] >= '2017-01-1') & (accepted['issue_d'] < '2018-01-1'), :]
accepted_2018      = accepted.loc[(accepted['issue_d'] >= '2018-01-1') & (accepted['issue_d'] < '2019-01-1'), :]

In [ ]:
dfs = [accepted_b_2015, accepted_2015, accepted_2016, accepted_2017, accepted_2018]

In [ ]:
apriori_columns_bac = apriori_columns.copy()

### Filling the apriori columns NA's for each dataset

In [ ]:
def fill_nas(df, apriori_columns):
    apriori = apriori_columns.copy()
    
    df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0) #almost all is 0
    
    null_sum = df[apriori].isnull().sum()
    p_null_sum = null_sum[null_sum > 0]

    # Droping observations with NA's that are in columns with low NA's
    columns_na = p_null_sum[p_null_sum/df.shape[0]<.01].index

    idx = df.loc[df[columns_na].isnull().any(axis=1), :].index
    df = df.drop(index=idx)

    null_sum = df[apriori].isnull().sum()
    p_null_sum = null_sum[null_sum > 0]

    # drop columns that have many NA's
    drop_na_columns = p_null_sum[p_null_sum/df.shape[0]>.5].index

    df.drop(columns = drop_na_columns, inplace = True)

    apriori = [col for col in apriori if col not in drop_na_columns] #update apriori

    null_sum = df[apriori].isnull().sum()
    p_null_sum = null_sum[null_sum > 0]

    # Fill emp_length NA's by random sampling proportionaly
    a = df['emp_length'].value_counts()/df['emp_length'].shape[0]*1000
    a = a.apply(lambda x: int(x))
    x = pd.Series(range(1, np.sum(a)+1))

    n = np.sum(df['emp_length'].isnull())

    x = x.sample(n, replace = True).reset_index()\
    .set_index(df.loc[df['emp_length'].isnull(), 'emp_length'].index)[0]

    df = df.fillna({'emp_length':pd.cut(x, bins = [0]+list(np.cumsum(a)), labels = a.index)})

    null_sum = df[apriori].isnull().sum()
    p_null_sum = null_sum[null_sum > 0]

    #Fill the rest of apriori columns NA's by the median of the sub_grade
    null_sum = df[apriori].isnull().sum()
    for col in null_sum[null_sum > 0].index:
        a  = df.groupby(['sub_grade'])[col].agg('median')
        df = df.fillna({col:df.loc[df[col].isnull(), 
                       ['sub_grade', col]].apply(lambda s: a[s[0]], axis = 1)})

    null_sum = df[apriori].isnull().sum()
    p_null_sum = null_sum[null_sum > 0]

    # removing all the non apriori columns with NA's, only keeping last_credit_pull_d
    df = df.fillna({'last_credit_pull_d':np.mean(df['last_credit_pull_d'])})

    columns_drop = df.isnull().sum()[df.isnull().sum() > 0].index
    df = df.drop(columns = columns_drop)

    df = df.reset_index(drop = True)
    
    return df, apriori

In [ ]:
clean_dfs = []
for df in dfs:
    clean_dfs.append(fill_nas(df, apriori_columns))

# Feature engeniering

In [ ]:
def loan_length(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

def calculate_duration(df):
    loan_months = []
    for end, start in zip(df['last_pymnt_d'], df['issue_d']):
        loan_months.append(loan_length(end, start))
        
    days_in_dt = df['last_pymnt_d'] - df['issue_d']
    
    df['duration_days'] = days_in_dt.dt.days
    df['duration_months'] = loan_months
    return df

def simplify_loan_status(df):
    paid = ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']

    df.loc[:, 'loan_status'] = df['loan_status'].apply(lambda s: 'FullyPaid' if s in paid  else
                                                                 'Current' if s=='Current' else
                                                                 'Defaulted')
    return df

def calculate_invst_return(df):
    df['invest_return_per'] = list(map(lambda x: round(x,2), 
                                          (df['total_pymnt'] - df['funded_amnt'])/df['funded_amnt']*100))
    df['invest_return']     = list(map(lambda x: round(x,2), df['total_pymnt'] - df['funded_amnt']))
    return df

def calculate_fico_mid(df):
    df['fico_mid'] = (df['fico_range_high'] + df['fico_range_low'])/2
    return df

def generate_features(df):
    df = calculate_duration(df)
    df = simplify_loan_status(df)
    df = calculate_invst_return(df)
    df = calculate_fico_mid(df)
    return df

In [ ]:
clean_fg_dfs = []
for df, _ in clean_dfs:
    clean_fg_dfs.append(generate_features(df))

In [ ]:
# Random permutation of the dataset
for i, df in enumerate(clean_fg_dfs):
    perm            = np.random.permutation(df.shape[0]) 
    clean_fg_dfs[i] = df.iloc[perm, :].reset_index(drop=True)

# Selecting featues

In [ ]:
post_loan_columns = ['verification_status', 'loan_status', 'out_prncp',
                     'total_pymnt','total_rec_prncp','total_rec_int',
                     'total_rec_late_fee', 'recoveries', 
                     'collection_recovery_fee', 'last_pymnt_d',
                     'last_pymnt_amnt','last_credit_pull_d', 
                     'last_fico_range_high','last_fico_range_low']

genereted_columns = ['duration_days','duration_months','invest_return_per',
                     'invest_return']
#Not sure about verification_status

#Remove from features
remove_columns = ['issue_d', 'id']


counting_columns = ['delinq_2yrs','inq_last_6mths', 'open_acc', 
                    'pub_rec', 'total_acc', 'collections_12_mths_ex_med', 
                    'acc_now_delinq', 'chargeoff_within_12_mths', 
                    'mort_acc','num_accts_ever_120_pd', 'num_actv_bc_tl', 
                    'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
                    'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
                    'num_tl_120dpd_2m', 'num_tl_30dpd','num_tl_90g_dpd_24m', 
                    'num_tl_op_past_12m', 'pub_rec_bankruptcies', 'tax_liens',
                    'acc_open_past_24mths', 'num_actv_rev_tl']


# Categorical
cat_columns = ['term', 'home_ownership', 'purpose', 'addr_state', 'initial_list_status']

#Ordinal
ordinal_columns = ['sub_grade', 'emp_length'] #grade is removed

#earliest_cr_line is a datetime column and was removed. The rest is the number of months since something
time_columns = ['mo_sin_old_rev_tl_op', 'mo_sin_rcnt_tl', 
                'mths_since_recent_bc','mo_sin_rcnt_rev_tl_op'] 

non_continuous_columns = counting_columns + cat_columns + ordinal_columns + time_columns
continuous_columns = ['loan_amnt','funded_amnt','int_rate',
                      'installment','annual_inc','dti',
                      'fico_mid', 'revol_bal',
                      'revol_util','tot_coll_amt',
                      'tot_cur_bal','total_rev_hi_lim',
                      'avg_cur_bal','bc_open_to_buy','bc_util',
                      'delinq_amnt','pct_tl_nvr_dlq',
                      'percent_bc_gt_75','tot_hi_cred_lim',
                      'total_bal_ex_mort','total_bc_limit',
                      'total_il_high_credit_limit']

feature_columns = continuous_columns + non_continuous_columns

keep_columns = remove_columns + feature_columns + post_loan_columns

ml_columns = feature_columns + ['loan_status']

In [ ]:
final_dfs  = []
for df in clean_fg_dfs:
    final_dfs.append(df.loc[:, keep_columns])

In [ ]:
model_dfs = []
for df in clean_fg_dfs:
    model_dfs.append(df.loc[:, ml_columns])

# Saving the new datasets

In [ ]:
model_dfs[0].to_csv('/Users/ivanpassoni/Google Drive/LendingClubData/ml datasets/accepted_b_2015_ml.csv', index = False)

for df, year in zip(model_dfs[1:], [2015, 2016, 2017, 2018]):
    df.to_csv(f'/Users/ivanpassoni/Google Drive/LendingClubData/ml datasets/accepted_{year}_ml.csv',
              index = False)

In [ ]:
final_dfs[0].to_csv('/Users/ivanpassoni/Google Drive/LendingClubData/accepted loans/accepted_b_2015.csv', index = False)

for df in final_dfs[1:]:
    df.to_csv('/Users/ivanpassoni/Google Drive/LendingClubData/accepted loans/accepted_' + str(df['issue_d'].max().year) + '.csv',
              index = False)

In [ ]:
# # To read the dataset
# parse_dates = ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d']

# accepted = pd.read_csv('/Users/ivanpassoni/Google Drive/LendingClubData/accepted_b_2015.csv',
#                        low_memory = False, parse_dates = parse_dates)